In [1]:
%autoreload 2 
%matplotlib inline

In [2]:
from __future__ import (print_function, unicode_literals, 
                        division, absolute_import)

In [3]:
import h5py
import pandas as pd
import numpy as np 

In [4]:
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

In [5]:
import healpy as hp

In [6]:
import extract_catalog as ext_cat
import get_gal_centroids as getg
import plot_clst_prop as plotClst
from IPython.html.widgets import interact

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [7]:
dataPath = "../../data/"

In [8]:
original_f = h5py.File(dataPath + 
                "Illustris-1_fof_subhalo_myCompleteHaloCatalog_00135.hdf5")

In [49]:
clstNo = 4

In [9]:
df = ext_cat.extract_clst(original_f, clstNo)

In [10]:
pos = [k for k in df.keys() if "SubhaloPos" in k]

# nsides of HEALpix are powers of 2

In [41]:
xi_array, phi_array = getg.angles_given_HEALpix_nsides(2)

# Start projection!

metadata needed 

In [112]:
df.keys()

Index([u'SubhaloGrNr', u'SubhaloLenType0', u'SubhaloLenType1', u'SubhaloLenType2', u'SubhaloLenType3', u'SubhaloLenType4', u'SubhaloLenType5', u'SubhaloMass', u'SubhaloMassType0', u'SubhaloMassType1', u'SubhaloMassType2', u'SubhaloMassType3', u'SubhaloMassType4', u'SubhaloMassType5', u'SubhaloPos0', u'SubhaloPos1', u'SubhaloPos2', u'U_band', u'B_band', u'V_band', u'K_band', u'g_band', u'r_band', u'i_band', u'z_band', u'SubhaloStellarPhotometricsMassInRad'], dtype='object')

In [42]:
df.shape[0]

7573

In [43]:
cut_kwargs={"DM_cut": 1e3, "star_cut": 1e2}

mask = getg.cut_reliable_galaxies(df, **cut_kwargs)

In [44]:
col = ["SubhaloPos{0}".format(i) for i in range(3)]

data = np.array(df[col][mask])

In [64]:
data_arr, xi_array, phi_array, los_axis = \
    getg.prep_data_with_cuts_and_proj(df, getg.cut_reliable_galaxies, 
                                      cut_kwargs, nside_pow=1, 
                                      verbose=True)

# of subhalos after the cut = 265


In [88]:
proj_no = 2
stuff = \
    map(lambda d, xi, phi: 
        getg.compute_KDE_peak_offsets(d, original_f, clstNo, xi=xi, 
                                  phi=phi, los_axis=los_axis),
        data_arr[:proj_no], xi_array[:proj_no], phi_array[:proj_no])

In [90]:
offset_list = [s[0] for s in stuff]
offset_R200_list = [s[1] for s in stuff]
fhat_list = [s[2] for s in stuff]

In [110]:
peak_df = getg.convert_dict_peaks_to_df(fhat_list, wt="None")

In [111]:
peak_df

,peaks_xcoords,peaks_ycoords,peaks_rowIx,peaks_colIx,peaks_dens,clstNo,phi,xi,los_axis,weights
0,-9.949375,46.142690,70,78,1.000000,0,0.785398,0.411138,2,None
1,-376.980629,1199.727221,54,122,0.115646,0,0.785398,0.411138,2,None
2,-514.617349,-923.917028,48,41,0.103416,0,0.785398,0.411138,2,None
0,-46.142691,-9.949378,72,70,1.000000,1,2.356194,0.411138,2,None
1,-1199.727231,-376.980642,28,54,0.115648,1,2.356194,0.411138,2,None
2,923.917036,-514.617366,109,48,0.103414,1,2.356194,0.411138,2,None


In [ ]:
getg.convert_dict_dens_to_h5(fhat_list, wt="None")